## Métodos de Otimização: Métodos de procura multidimensional

### Método de Newton - Diferenças finitas

__Imports:__

In [8]:
from mpl_toolkits import mplot3d
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

__Função Avaliada:__

In [9]:
def f(x,nFunc,nVar):
    f = open("var.dat", "w")
    f.write(str(nFunc)+"\n")
    f.write(str(nVar)+"\n")
    for i in range(0, len(x)):
        f.write(str(x[i])+'\n')
    f.close()
    
    result = os.system("funcao.exe")
    
    f = open("obj.dat", "r")
    u = float(f.read())
    f.close()
    return u

__Método:__

In [10]:
def grad(x):
    epsilon = 1.e-10
    nFunc = 5; Qvar = 2
    
    for i in range(0,len(x)):
        xtmp=x.copy()
        xtmp[i]=xtmp[i]+epsilon
        G[i]=(f(xtmp,nFunc,Qvar)-f(x,nFunc,Qvar))/epsilon
    
    return G

In [11]:
def InvHess(x):
    epsilon = 1.e-10
    nFunc = 5; Qvar = 2
    Hinv = np.zeros((len(x),len(x)), dtype='float')

    for i in range(0,len(x)):
        xtmp=x.copy(); xtmp2=x.copy()
        xtmp[i]=xtmp[i]+epsilon
        xtmp2[i]=xtmp[i]-epsilon
        xtmp3 = [xtmp[i], xtmp2[i]]
        D2[i]=(f(xtmp,nFunc,Qvar)-2*f(x,nFunc,Qvar)+f(xtmp2,nFunc,Qvar))/(epsilon**2)
        D2_partial[i]= (1./epsilon)*(((f(xtmp,nFunc,Qvar)-f(xtmp3,nFunc,Qvar))/epsilon)\
            -((f(x,nFunc,Qvar)-f(xtmp2,nFunc,Qvar))/epsilon))

    for i in range(0,len(x)):
        for j in range(0,len(x)):
            if i==j:
                Hinv[i,j] = D2[i]
            else:
                Hinv[i,j] = D2_partial[i]

    Hinv = np.linalg.inv(Hinv)
    return(Hinv)

In [12]:
def InvHess(x):
    epsilon = 1.e-10
    nFunc = 5; Qvar = 2
    Hinv = np.zeros((len(x),len(x)), dtype='float')

    for i in range(0,len(x)):
        xtmp=x.copy(); xtmp2=x.copy()
        xtmp[i]=xtmp[i]+epsilon
        xtmp2[i]=xtmp[i]-epsilon
        xtmp3 = [xtmp[i], xtmp2[i]]
        D2[i]=(f(xtmp,nFunc,Qvar)-2*f(x,nFunc,Qvar)+f(xtmp2,nFunc,Qvar))/(epsilon**2)
        D2_partial[i]= (1./epsilon)*(((f(xtmp,nFunc,Qvar)-f(xtmp3,nFunc,Qvar))/epsilon)\
            -((f(x,nFunc,Qvar)-f(xtmp2,nFunc,Qvar))/epsilon))

    for i in range(0,len(x)):
        for j in range(0,len(x)):
            if i==j:
                Hinv[i,j] = D2[i]
            else:
                Hinv[i,j] = D2_partial[i]

    Hinv = np.linalg.inv(Hinv)
    return(Hinv)

In [30]:
def InvHess(x):
    epsilon = 1.e-10
    nFunc = 5; Qvar = 2
    Hinv=np.zeros((2,2), dtype='float')
    F = f(x,nFunc,Qvar)
    for i in range(0,len(x)):
        xtmp=x.copy()
        xtmp[i]=x[i]+epsilon; Fx = f(xtmp,nFunc,Qvar)
        for j in range(0,len(x)):
            if i==j:
                xtmp2=x.copy()
                xtmp2[i]=x[i]-epsilon; Fy = f(xtmp2,nFunc,Qvar)
                Hinv[i,i] = (Fx-2*F+Fy)/(epsilon**2)
            else:
                xtmp3 = xtmp.copy()
                xtmp3[j]=x[j]+epsilon; Fz = f(xtmp3,nFunc,Qvar)
                xtmp3[i]=x[i]; Fh = f(xtmp3,nFunc,Qvar)
                Hinv[i,j] = (Fz-Fx-Fh+F)/(epsilon**2)
                
    if np.linalg.det(Hinv) == 0:
        Hinv=np.identity(2, dtype='float')
    else:
        Hinv = np.linalg.inv(Hinv)

    return(Hinv)

In [35]:
def InvHess(x):
    epsilon = 1.e-10
    nFunc = 5; Qvar = 2
    Hinv=np.zeros((2,2), dtype='float')
    initial = np.array((2,2), dtype='float')
    n = len(x)
    for i in range(0,len(x)):
        for j in range(0,len(x)):
            ei = np.zeros(n)
            ei[i] = 1
            ej = np.zeros(n)
            ej[j] = 1
            f1 = f(x+(epsilon * ei)+(epsilon * ej),nFunc,Qvar)
            f2 = f(x+(epsilon * ei)-(epsilon * ej),nFunc,Qvar)
            f3 = f(x-(epsilon * ei)+(epsilon * ej),nFunc,Qvar)
            f4 = f(x-(epsilon * ei)-(epsilon * ej),nFunc,Qvar)
            numdiff = (f1-f2-f3+f4)/(4*epsilon*epsilon)
            Hinv[i,j] = numdiff
                
    if np.linalg.det(Hinv) == 0:
        Hinv=np.identity(2, dtype='float')
        print('Flag: Matriz Singular')
    else:
        Hinv = np.linalg.inv(Hinv)

    return(Hinv)

In [36]:
nFunc = 5; Qvar = 2

Hinv=np.zeros((2,2), dtype='float')
G=np.zeros(2, dtype='float')
D2=np.zeros(2, dtype='float')
D2_partial=np.zeros(2, dtype='float')
xnew=np.zeros(2, dtype='float')
xold=np.zeros(2, dtype='float')
kmax=10; k=0
xold[0]=3; xold[1]=2; F0 = f(xold,nFunc,Qvar)
layout = "{0:.5f}"

while k < kmax:
    k += 1
    # Calculo Gradiente
    G = grad(xold)

    # calcula a inversa do Hessiano
    Hinv = InvHess(xold)
    print(Hinv)
    
    # Processo iterativo
    xnew=xold-np.matmul(Hinv,G)
    
    #Atualiza xold
    xold = xnew.copy()
    
    #Calculo da função
    F = f(xnew,nFunc,Qvar)
    print(layout.format(xnew[0])+" "+layout.format(xnew[1])+" "+layout.format(F))

Flag: Matriz Singular
[[1. 0.]
 [0. 1.]]
4.98863 2.77602 14.87107
Flag: Matriz Singular
[[1. 0.]
 [0. 1.]]
-5.15737 -0.22498 333.44487
Flag: Matriz Singular
[[1. 0.]
 [0. 1.]]
108.01219 35.51476 1704936.09527
[[-8.68866947e-11 -8.49119971e-11]
 [-8.49119971e-11 -8.68866947e-11]]
108.01220 35.51476 1704936.47560
Flag: Matriz Singular
[[1. 0.]
 [0. 1.]]
-68590.99752 2636.23305 369523100717779904.00000
Flag: Matriz Singular
[[1. 0.]
 [0. 1.]]
24319999931409.00391 2636.23305 5838148751324380908017706652253467176694561642643456.00000
Flag: Matriz Singular
[[1. 0.]
 [0. 1.]]
24319999931409.00391 2636.23305 5838148751324380908017706652253467176694561642643456.00000
Flag: Matriz Singular
[[1. 0.]
 [0. 1.]]
24319999931409.00391 2636.23305 5838148751324380908017706652253467176694561642643456.00000
Flag: Matriz Singular
[[1. 0.]
 [0. 1.]]
24319999931409.00391 2636.23305 5838148751324380908017706652253467176694561642643456.00000
Flag: Matriz Singular
[[1. 0.]
 [0. 1.]]
24319999931409.00391 2636.23